In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer
import torch
import copy

/data3/home/yaswanthk/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Splitting data into Train and Val

In [2]:
df = pd.read_excel("ArithOpsTrain.xlsx", header=1)  # header=1 indicates that the second row should be used as the column names

# Drop the first column
df.drop(df.columns[0], axis=1, inplace=True)

df = df.sample(frac=1)

df.head()

,Description,Question,Equation,Input Numbers,Output
110,after the final bell rang buddy ran to the sch...,how many students are on the bus ride home ?,+ number0 number1,36 27,63.0
675,paul was reading through his favorite book ser...,how many books would he have read through afte...,* number0 number1,4 5,20.0
978,there are number0 students and number1 apples ...,how many does each student get ?,/ number1 number0,43 1720 9,40.0
597,there were number0 red roses and number1 white...,how many red roses did she cut ?,- number2 number0,8 2 14,6.0
60,zach scored number0 points in the football gam...,how many more points did zach score ?,- number0 number1,42 21,21.0


In [3]:
def fun(equation):
    return equation.count('+') + equation.count('-') + equation.count('/') + equation.count('*') + 1
df['numbers'] = df['Equation'].apply(fun)
df.head()

,Description,Question,Equation,Input Numbers,Output,numbers
110,after the final bell rang buddy ran to the sch...,how many students are on the bus ride home ?,+ number0 number1,36 27,63.0,2
675,paul was reading through his favorite book ser...,how many books would he have read through afte...,* number0 number1,4 5,20.0,2
978,there are number0 students and number1 apples ...,how many does each student get ?,/ number1 number0,43 1720 9,40.0,2
597,there were number0 red roses and number1 white...,how many red roses did she cut ?,- number2 number0,8 2 14,6.0,2
60,zach scored number0 points in the football gam...,how many more points did zach score ?,- number0 number1,42 21,21.0,2


In [4]:
df['numbers'].value_counts()

numbers
2    754
3    225
Name: count, dtype: int64

In [5]:
# # Split the DataFrame into a training set (70%) and a testing set (30%)
# train_df = df.sample(frac=0.7, random_state=1)  # You can change the random_state for reproducibility
# val_df = df.drop(train_df.index)

# # Save the training set to a CSV file
# train_df.to_csv('training_data.csv', index=False)

# # Save the testing set to a CSV file
# val_df.to_csv('val_data.csv', index=False)

In [6]:
train_df = pd.read_csv("training_data.csv")
val_df = pd.read_csv('val_data.csv')

train_df.head()

,Description,Question,Equation,Input Numbers,Output,numbers
0,allen shiela 's brother likes to play with blo...,how many colors did shiela use ?,/ number0 number1,49 7,7.00,2
1,isabel received number0 dollars for her birthd...,how many of the toys could she buy ?,/ number0 number1,14 2,7.00,2
2,nick saved $ number0 . if nick saved $ number1...,how much did lee save ?,- number0 number1,68.50 25.43,43.07,2
3,if lewis earns $ number0 every week during the...,how much money does he earn during harvest sea...,* number0 number1,1367.00 5,6835.00,2
4,amy uploaded number0 pics to facebook . if she...,how many photos were in each album ?,/ number0 number1,180 9,20.00,2


## Model training

In [7]:
train_descriptions = list(train_df['Description'].values)
train_questions = list(train_df['Question'].values)
train_equations = list(train_df['Equation'].values)

val_descriptions = list(val_df['Description'].values)
val_questions = list(val_df['Question'].values)
val_equations = list(val_df['Equation'].values)

In [8]:
# Initialize tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

In [9]:
# # Max description+question token length
# source_max_len = 0
# for i in range(len(descriptions)):
#     text = descriptions[i]+" "+questions[i]
#     tokens = tokenizer(text)['input_ids']
#     source_max_len = max(source_max_len,len(list(tokens)))
# source_max_len
    

In [10]:
# # Max equation token length
# target_max_len = 0
# for i in range(len(descriptions)):
#     text = equations[i]
#     tokens = tokenizer(text)['input_ids']
#     target_max_len = max(target_max_len,len(list(tokens)))
# target_max_len
    

In [11]:
class MathEquationDataset(Dataset):
    def __init__(self, descriptions, questions, equations, tokenizer, src_max_length=100,tgt_max_len = 15):
        self.descriptions = descriptions
        self.questions = questions
        self.equations = equations
        self.tokenizer = tokenizer
        self.src_max_length = src_max_length
        self.tgt_max_length = tgt_max_len

    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, idx):
        src_text = self.descriptions[idx] + " " + self.questions[idx]
        tgt_text = self.equations[idx]
        
        encoding = self.tokenizer(src_text, return_tensors='pt', max_length=self.src_max_length, padding='max_length', truncation=True)
        target = self.tokenizer(tgt_text, return_tensors='pt', max_length=self.tgt_max_length, padding='max_length', truncation=True)
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': target['input_ids'].flatten()
        }


In [12]:
BATCH_SIZE = 16
source_max_len = 100
target_max_len = 10

# Initialize the Dataset
train_dataset = MathEquationDataset(train_descriptions, train_questions, train_equations, tokenizer,src_max_length=source_max_len,tgt_max_len=target_max_len)
val_dataset = MathEquationDataset(val_descriptions, val_questions, val_equations, tokenizer,src_max_length=source_max_len,tgt_max_len=target_max_len)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Train_size == ",len(train_dataset))
print("Val_size == ",len(val_dataset))

Train_size ==  685
Val_size ==  294


In [13]:
train_iter = iter(train_loader)
batch = next(train_iter)
batch

{'input_ids': tensor([[    0,   119, 10147,  ...,     1,     1,     1],
         [    0, 10010,  2867,  ...,     1,     1,     1],
         [    0,   102, 10305,  ...,     1,     1,     1],
         ...,
         [    0, 33243,   342,  ...,     1,     1,     1],
         [    0, 23018,   829,  ...,     1,     1,     1],
         [    0, 18287,   324,  ...,     1,     1,     1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[   0, 2744,  346,  288,  346,  134,    2,    1,    1,    1],
         [   0, 3226,  346,  288,  346,  134,    2,    1,    1,    1],
         [   0,   12,  346,  288,  346,  134,    2,    1,    1,    1],
         [   0,   12,  346,  288,  346,  134,    2,    1,    1,    1],
         [   0,   73,  346,  134,  346,  288,    2,    1,    1,    1],
        

In [14]:
ids = tokenizer('robyn and lucy are members of their village s girl scout troop . during weekends and some weekdays they go around selling cookies in the neighborhood . they have a week before the month ends and they are doing their best to get a badge from selling cookies . working overtime robyn sold number0 packs of cookies while lucy sold number1 how	many packs of cookies were they able to sell that day ?',max_length=100, padding='max_length', truncation=True)['input_ids']
tokenizer.decode(ids)

'<s>robyn and lucy are members of their village s girl scout troop. during weekends and some weekdays they go around selling cookies in the neighborhood. they have a week before the month ends and they are doing their best to get a badge from selling cookies. working overtime robyn sold number0 packs of cookies while lucy sold number1 how\tmany packs of cookies were they able to sell that day?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [15]:
tokenizer.decode(batch['input_ids'][3])

'<s>at the edge of the forest an anthill is blocking the way out. in order to pass through he needs to help the ants gather food. if the ants need number0 grains of food and they already have number1 how many more grains are needed to be gathered?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [16]:
tokenizer.decode(batch['labels'][3])

'<s>- number0 number1</s><pad><pad><pad>'

In [17]:
device = 'cuda:5'

In [18]:
from transformers import BartForConditionalGeneration, AdamW

# model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
model.to(device)  # or 'cpu' if you are not using a GPU


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [19]:
output = model(input_ids=batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device),labels = batch['labels'].to(device))

In [20]:
output['logits'].shape

torch.Size([16, 10, 50265])

In [21]:
output['loss']

tensor(8.8609, device='cuda:5', grad_fn=<NllLossBackward0>)

In [22]:
def eval(model,val_loader,val_dataset):
    model.eval()
    total_val_loss = 0
    total_true = []
    total_pred = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)


            pred_tokens = model.generate(input_ids)
            pred_output = [tokenizer.decode(output_id, skip_special_tokens=True) for output_id in pred_tokens]
            true_output = [tokenizer.decode(output_id, skip_special_tokens=True) for output_id in labels]

            total_true.extend(true_output)
            total_pred.extend(pred_output)

            # print("True == ",true_output)
            # print("Predicted == ",pred_output)
            # print("\n\n")
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss = outputs.loss
            total_val_loss += val_loss.item()
        
        avg_val_loss = total_val_loss / len(val_loader)

        matching_strings = [a for a, b in zip(total_true, total_pred) if a == b]

        # Get the count of matching strings
        acc = len(matching_strings)/len(val_dataset)
        # print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {acc:.4f}")
        
    return avg_val_loss,acc,total_true,total_pred


In [23]:
optimizer = AdamW(model.parameters(), lr=5e-5)
train_loss_stat = []
val_loss_stat = []
best_val_loss = 0.5
best_model_wts = copy.deepcopy(model.state_dict())
best_accuracy = 0.0
best_epoch = 0

/data3/home/yaswanthk/.local/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    print(f"Epoch {epoch+1}/{num_epochs}")
    total_loss = 0
    
    for i,batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i% 10== 0:
            model.eval()
            avg_val_loss,val_acc,total_true,total_pred = eval(model,val_loader,val_dataset)
            model.train()
            
            avg_train_loss = total_loss / len(train_loader)

            if val_acc > best_accuracy:
                best_model_wts = copy.deepcopy(model.state_dict())
                best_accuracy = val_acc
                best_epoch = epoch+1
                print("--------- Accuracy Noted ----------")
            train_loss_stat.append(avg_train_loss)
            val_loss_stat.append(avg_val_loss)

            print(f"Iter : {i}, Training Loss: {avg_train_loss:.4f}, Validation Loss : {avg_val_loss:.4f}, Val_Accuracy : {val_acc:.4f}")
    
    model.eval()
    _,train_acc,_,_ = eval(model,train_loader,train_dataset)
    print(f"Train_Accuracy : {train_acc:.4f}")    
    model.train()       


print(f"Best_acc : {best_accuracy:.4f}, Best_epoch : {best_epoch}")
model.load_state_dict(best_model_wts)

Epoch 1/10


/data3/home/yaswanthk/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Iter : 0, Training Loss: 0.1885, Validation Loss : 5.8523, Val_Accuracy : 0.0000
--------- Accuracy Noted ----------
Iter : 10, Training Loss: 1.1651, Validation Loss : 3.0790, Val_Accuracy : 0.1803
--------- Accuracy Noted ----------
Iter : 20, Training Loss: 1.8475, Validation Loss : 2.0637, Val_Accuracy : 0.2075
--------- Accuracy Noted ----------
Iter : 30, Training Loss: 2.2972, Validation Loss : 1.3394, Val_Accuracy : 0.2211
Iter : 40, Training Loss: 2.6374, Validation Loss : 1.0572, Val_Accuracy : 0.1837
Train_Accuracy : 0.2540
Epoch 2/10
Iter : 0, Training Loss: 0.0259, Validation Loss : 0.9199, Val_Accuracy : 0.2177
--------- Accuracy Noted ----------
Iter : 10, Training Loss: 0.2588, Validation Loss : 0.6949, Val_Accuracy : 0.2279
--------- Accuracy Noted ----------
Iter : 20, Training Loss: 0.4502, Validation Loss : 0.5031, Val_Accuracy : 0.2857
--------- Accuracy Noted ----------
Iter : 30, Training Loss: 0.5818, Validation Loss : 0.4179, Val_Accuracy : 0.3299
Iter : 40, Tr

<All keys matched successfully>

In [25]:
avg_val_loss,acc,total_true,total_pred = eval(model,val_loader,val_dataset)
print(f"Validation Loss: {avg_val_loss:.4f}, Val_Accuracy: {acc:.4f}")


Validation Loss: 0.1528, Val_Accuracy: 0.6395


In [26]:
unmatched_true = []
unmatched_pred = []
for true,pred in zip(total_true,total_pred):
    if true != pred:
        unmatched_true.append(true)
        unmatched_pred.append(pred)
model_df = pd.DataFrame({'True': unmatched_true, 'Pred': unmatched_pred})
model_df

,True,Pred
0,/ number2 number1,/ - number0 number1 number2
1,- - number0 number1 number2,+ - number0 number1 number2
2,+ - number0 number1 number3,- - number0 number3 number3
3,- number1 number0,- number0 number1
4,+ number0 number1,- number0 number1
...,...,...
101,/ - number0 number1 number2,- - number0 number1 number2
102,+ * number1 number2 number0,* - number0 number1 number2
103,- - number0 number1 number2,- - number1 number2 number0
104,* number0 number1,/ number0 number1


In [27]:
torch.save(model.state_dict(), f'checkpoint_{20}_acc_0.77.pth')

In [28]:
val_iter = iter(val_loader)
batch = next(val_iter)
pred_tokens = model.generate(batch['labels'].to(device))
pred_output = [tokenizer.decode(output_id, skip_special_tokens=True) for output_id in pred_tokens]
true_output = [tokenizer.decode(output_id, skip_special_tokens=True) for output_id in batch['labels']]
print("True == ",true_output)
print("Pred == ",pred_output)

True ==  ['- number0 number1', '/ number1 number0', '* number0 number1', '/ number2 number1', '+ number0 number1', '+ + number0 number1 number2', '- number0 number2', '- - number0 number1 number2', '+ - number0 number1 number3', '- number1 number0', '+ number0 number1', '+ number0 number1', '+ number0 number1', '* - number1 number2 number0', '- number0 number3', '- number1 number0']
Pred ==  ['- number0 number1', '- number0 number1', '- number0 number1', '/ number2 number1', '+ number0 number1', '+ + number0 number1 number2', '- - number0 number1 number2', '- - number0 number1 number2', '+ number0 number3', '- number0 number1', '+ number0 number1', '+ number0 number1', '+ number0 number1', '- - number1 number2 number0', '- number0 number3', '- number0 number1']


/data3/home/yaswanthk/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## Loading the saved model

In [29]:
from transformers import BartForConditionalGeneration, AdamW
# model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
# temp = torch.load('checkpoint_20_acc_0.77.pth')
model.load_state_dict(torch.load('checkpoint_20_acc_0.77.pth'))
model.eval()
model.to(device)  # or 'cpu' if you are not using a GPU

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [30]:
avg_val_loss,acc,total_true,total_pred = eval(model,train_loader,train_dataset)
print(f"Train Loss: {avg_val_loss:.4f}, Train_Accuracy: {acc:.4f}")


Train Loss: 0.0407, Train_Accuracy: 0.8949


In [31]:
avg_val_loss,acc,total_true,total_pred = eval(model,val_loader,val_dataset)
print(f"Val Loss: {avg_val_loss:.4f}, Val_Accuracy: {acc:.4f}")


Val Loss: 0.1528, Val_Accuracy: 0.6395


In [32]:
unmatched_true = []
unmatched_pred = []
for true,pred in zip(total_true,total_pred):
    if true != pred:
        unmatched_true.append(true)
        unmatched_pred.append(pred)
model_df = pd.DataFrame({'True': unmatched_true, 'Pred': unmatched_pred})
model_df

,True,Pred
0,/ number2 number1,/ - number0 number1 number2
1,- - number0 number1 number2,+ - number0 number1 number2
2,+ - number0 number1 number3,- - number0 number3 number3
3,- number1 number0,- number0 number1
4,+ number0 number1,- number0 number1
...,...,...
101,/ - number0 number1 number2,- - number0 number1 number2
102,+ * number1 number2 number0,* - number0 number1 number2
103,- - number0 number1 number2,- - number1 number2 number0
104,* number0 number1,/ number0 number1


In [33]:
def generate_equation(model,sentence):
    model.eval()
    inputs = tokenizer(sentence, return_tensors="pt", max_length=100, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model.generate(**inputs)
    decoded_output = [tokenizer.decode(output_id, skip_special_tokens=True) for output_id in outputs]
    return outputs,decoded_output


In [34]:
description = "the kids from oakwood elementary school are visiting a bird zoo for their field trip . to get to the bird zoo from the school the kids have to ride some buses . if there are number0 buses and each bus has number1 adult supervisors to guide the children	how many supervisors are there in total ?"
outputs,predicted_equation = generate_equation(model,description)
print(predicted_equation)


['* number0 number1']


/data3/home/yaswanthk/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [35]:
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[   2,    0, 3226,  346,  288,  346,  134,    2]], device='cuda:5')
</s><s>* number0 number1</s>


In [36]:
tgt = "* number0 number1"
ids = tokenizer(tgt)['input_ids']
print(ids)
print(tokenizer.decode(ids))

[0, 3226, 346, 288, 346, 134, 2]
<s>* number0 number1</s>


In [37]:
tokenizer.decode(outputs[0])

'</s><s>* number0 number1</s>'

## Test set results

In [38]:
def evaluate_prefix(expression, operands):
    # Split the expression into tokens
    tokens = expression.split()

    # Define a stack to store operands
    stack = []

    # Iterate through the tokens in reverse order (as it's a prefix expression)
    for token in reversed(tokens):
        try:
            if token.startswith('number'):
                # If the token starts with 'number', use it to index into the 'operands' list
                operand_index = int(token[6:])  # Extract the index from the token
                if 0 <= operand_index < len(operands):
                    stack.append(float(operands[operand_index]))
                else:
                    raise ValueError("Invalid operand index: " + str(operand_index))
            elif token in '+-*/':
                # If the token is an operator, pop two operands from the stack and apply the operator
                operand1 = stack.pop()
                operand2 = stack.pop()
                if token == '+':
                    stack.append(operand1 + operand2)
                elif token == '-':
                    stack.append(operand1 - operand2)
                elif token == '*':
                    stack.append(operand1 * operand2)
                elif token == '/':
                    if operand2 == 0:
                        raise ValueError("Division by zero")
                    stack.append(operand1 / operand2)
            else:
                raise ValueError("Invalid token: " + token)
        except (ValueError, IndexError):
            return None  # Handle the error and return None

    if len(stack) != 1:
        return None  # Return None for any errors

    return stack[0]

# Example usage:
expression = '+ * number0 number1 number2'
operands = [1, 3, 5]
result = evaluate_prefix(expression, operands)

if result is not None:
    print("Result:", result)
else:
    print("Error: Invalid expression")


Result: 8.0


In [39]:
test_df = pd.read_excel('ArithOpsTestDataWithoutOutput.xlsx')
test_df.head()

,Description,Question,Input Numbers
0,number0 red apples and number1 green apples ar...,how many apples are in the basket ?,7 2
1,ellen has number0 more balls than marin . mari...,how many balls does ellen have ?,6 9
2,janet has number0 oranges and sharon has numbe...,how many oranges do janet and sharon have toge...,9 7
3,allan brought number0 balloons and jake brough...,how many balloons did allan and jake have in t...,2 4
4,adam has number0 more apples than jackie . jac...,how many apples does adam have ?,5 9


In [40]:
# true = list(test_df['Output'].values)
pred = []
ind = 0
for index, row in test_df.iterrows():
    ind+=1
    if ind%50 == 0:
        print(ind)
    input = row['Description']+" "+row['Question']
    _,predicted_equation = generate_equation(model,input)
    predicted_equation = predicted_equation[0]
    pred_output = evaluate_prefix(predicted_equation, row['Input Numbers'].split())
    pred.append(pred_output)
     

/data3/home/yaswanthk/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


50
100
150
200


In [41]:
df = pd.DataFrame(pred)
df.to_excel('output.xlsx', index=False, header=False)
df.head()

,0
0,9.0
1,-3.0
2,2.0
3,6.0
4,14.0


In [42]:
df = pd.read_excel('ArithOpsTestDataOnlyOutput.xlsx')
true = list(df['Output'].values)
df.head()

,Output
0,9.0
1,15.0
2,16.0
3,6.0
4,14.0


In [43]:
correct = 0
for a,b in zip(true,pred):
    if a==b:
        correct+=1
correct/len(true)

0.6470588235294118

In [44]:
correct

154